In [1]:
!pip install PyMySQL
!pip install mysql-connector-python
!pip install sqlalchemy

In [ ]:
# DATA PROJECT 1. Jasmine Tang, Megan Le, Gabby Cordelli, Sam Remmey

In [69]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import pymongo
import datetime
import json

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [55]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "ds2002"
pwd = "UVA!1819"
db_name = "chinook"

In [21]:
# IMPORTING DATA FROM SQL

In [15]:
# General function for retrieving df from sql:
def get_sqldf(host_name, user_id, pwd, db_name, sqlQuery):
    try:
        conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
        df = pd.read_sql(sqlQuery, conn)
        conn.close()
        return df.head()
    except:
        print("Error. Check your connection.")

In [116]:
# invoice table
df_invoice = get_sqldf(host_name, user_id, pwd, db_name, ("SELECT * FROM invoice order by CustomerId;"))
df_invoice.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,98,1,2010-03-11,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,3.98
1,121,1,2010-06-13,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,3.96
2,143,1,2010-09-15,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,5.94
3,195,1,2011-05-06,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,0.99
4,316,1,2012-10-27,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,1.98


In [117]:
# customer table
df_customer = get_sqldf(host_name, user_id, pwd, db_name, ("SELECT * FROM customer order by CustomerId;"))
df_customer.head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [123]:
# create the invoice table (FACT TABLE)
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
df_invoice_fct = pd.read_sql("SELECT * FROM invoice order by InvoiceDate;", conn)
conn.close()
df_invoice_fct.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86


In [22]:
# IMPORTING DATA FROM MONGO

In [82]:
mysql_uid = "ds2002"
mysql_pwd = "UVA!1819"
atlas_cluster_name = "cluster0.kqsmdw7"
atlas_default_dbname = "local"
atlas_user_name = "gec2tp"
atlas_password = "DogWood#2002"
conn_str = {"local" : f"mongodb://localhost:27017/",
    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net"}
src_dbname = "chinook"
dst_dbname = "chinook_dw"
print(f"Local Connection String: {conn_str['local']}")

Local Connection String: mongodb://localhost:27017/


In [83]:
def get_mongodf(connect_str, db_name, collection, query):
    try:
        '''Create a connection to MongoDB'''
        client = pymongo.MongoClient(connect_str)
    
        '''Query MongoDB, and fill a python list with documents to create a dataframe'''
        db = client[db_name]
        df = pd.DataFrame(list(db[collection].find(query)))
        df.drop(['_id'], axis=1, inplace=True)
        client.close()
        return df
    except:
        print('Error: failure to get dataframe in MongoDB')

In [94]:
client = pymongo.MongoClient(conn_str["atlas"])
db = client[src_dbname]

# Appending the 'data' directory
data_dir = os.path.join("C:\\Users\\ds2002admin\\Desktop") 

In [95]:
# track table
json_files = {"track" : 'Track.json'}

for file in json_files:
    try:
        db.drop_collection(file)
        json_file = os.path.join(data_dir, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
            print(f"{file} was successfully loaded.")
    except:
        print('Error. Failure to retrieve JSON file.')
        
client.close() 

Collection(Database(MongoClient(host=['ac-jg6lcfb-shard-00-01.kqsmdw7.mongodb.net:27017', 'ac-jg6lcfb-shard-00-00.kqsmdw7.mongodb.net:27017', 'ac-jg6lcfb-shard-00-02.kqsmdw7.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-tm12uj-shard-0', tls=True), 'chinook'), 'track') was successfully loaded.


In [102]:
query = {}
collection = "track"

df_track = get_mongo_dataframe(conn_str['atlas'], src_dbname, collection, query) 
df_track.head(2)

,__href,MediaType,Genre,Album,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,PlaylistTrack,InvoiceLine
0,/db/Chinook/Track/TrackId/1.json,{'__href': '/db/Chinook/Track/TrackId/1/MediaT...,{'__href': '/db/Chinook/Track/TrackId/1/Genre....,{'__href': '/db/Chinook/Track/TrackId/1/Album....,1,For Those About To Rock (We Salute You) - TEST,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,{'__href': '/db/Chinook/Track/TrackId/1/Playli...,{'__href': '/db/Chinook/Track/TrackId/1/Invoic...
1,/db/Chinook/Track/TrackId/2.json,{'__href': '/db/Chinook/Track/TrackId/2/MediaT...,{'__href': '/db/Chinook/Track/TrackId/2/Genre....,{'__href': '/db/Chinook/Track/TrackId/2/Album....,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,{'__href': '/db/Chinook/Track/TrackId/2/Playli...,{'__href': '/db/Chinook/Track/TrackId/2/Invoic...


In [ ]:
# IMPORTING DATA FROM CSV

In [37]:
# artist table
df_artist = pd.read_csv("C:\\Users\\ds2002admin\\Downloads\\Artist.csv")
df_artist.head(5)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [38]:
# album table
df_album = pd.read_csv("C:\\Users\\ds2002admin\\Downloads\\Album.csv")
df_album.head(5)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [5]:
# RENAMING AND DELETING COLUMNS

In [40]:
df_artist.rename(columns={"ArtistId":"Artist_Key"}, inplace=True)
df_artist.head()

,Artist_Key,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [42]:
df_album.rename(columns={"AlbumId":"Album_Key","ArtistId":"Artist_Key"}, inplace=True)
df_album.head()

,Album_Key,Title,Artist_Key
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [118]:
df_invoice.rename(columns={"InvoiceId":"Invoice_Key","CustomerId":"Customer_Key"}, inplace=True)
df_invoice.drop(["BillingAddress","BillingState"],axis=1, inplace=True)
df_invoice.head()

,Invoice_Key,Customer_Key,InvoiceDate,BillingCity,BillingCountry,BillingPostalCode,Total
0,98,1,2010-03-11,São José dos Campos,Brazil,12227-000,3.98
1,121,1,2010-06-13,São José dos Campos,Brazil,12227-000,3.96
2,143,1,2010-09-15,São José dos Campos,Brazil,12227-000,5.94
3,195,1,2011-05-06,São José dos Campos,Brazil,12227-000,0.99
4,316,1,2012-10-27,São José dos Campos,Brazil,12227-000,1.98


In [119]:
df_customer.rename(columns={"CustomerId":"Customer_Key"}, inplace=True)
df_customer.drop(["Address","State","City","Country","PostalCode"], axis=1, inplace=True)
df_customer.head()

,Customer_Key,FirstName,LastName,Company,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [103]:
df_track.drop(['__href', 'MediaType', 'Genre', 'Album','PlaylistTrack','InvoiceLine'], axis=1, inplace=True)
df_track.rename(columns={"TrackId":"Track_Key","AlbumId":"Album_Key","MediaTypeId":"MediaType_Key","GenreId":"Genre_Key"},inplace=True)
df_track.head()

,Track_Key,Name,Album_Key,MediaType_Key,Genre_Key,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You) - TEST,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [ ]:
# MERGING TABLES TO MAKE DIMENSION TABLES

In [8]:
# merging df_customer and df_invoice

In [120]:
dim_customers = pd.merge(df_invoice, df_customer, on = "Customer_Key", how = "inner")
dim_customers.head()

,Invoice_Key,Customer_Key,InvoiceDate,BillingCity,BillingCountry,BillingPostalCode,Total,FirstName,LastName,Company,Phone,Fax,Email,SupportRepId
0,98,1,2010-03-11,São José dos Campos,Brazil,12227-000,3.98,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,121,1,2010-06-13,São José dos Campos,Brazil,12227-000,3.96,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,143,1,2010-09-15,São José dos Campos,Brazil,12227-000,5.94,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,195,1,2011-05-06,São José dos Campos,Brazil,12227-000,0.99,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
4,316,1,2012-10-27,São José dos Campos,Brazil,12227-000,1.98,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3


In [ ]:
# merging df_artist and df_album

In [65]:
dim_artstats = pd.merge(df_album, df_artist, on="Artist_Key", how="inner")
dim_artstats.head()

,Album_Key,Title,Artist_Key,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,4,Let There Be Rock,1,AC/DC
2,2,Balls to the Wall,2,Accept
3,3,Restless and Wild,2,Accept
4,5,Big Ones,3,Aerosmith


In [ ]:
# merging df_album and df_track

In [105]:
dim_alb_track = pd.merge(df_track,df_album,on="Album_Key", how="inner")
dim_alb_track.head()

,Track_Key,Name,Album_Key,MediaType_Key,Genre_Key,Composer,Milliseconds,Bytes,UnitPrice,Title,Artist_Key
0,1,For Those About To Rock (We Salute You) - TEST,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,For Those About To Rock We Salute You,1
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,For Those About To Rock We Salute You,1
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,For Those About To Rock We Salute You,1
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,For Those About To Rock We Salute You,1


In [ ]:
# LOADING DIMENSION TABLES BACK INTO SQL

In [10]:
# inserting dim_customers into chinook_dw in sql and declaring pk

In [49]:
dw_name = 'chinook_dw'

In [121]:
conn_string = f'mysql+pymysql://{user_id}:{pwd}@{host_name}/{dw_name}'
sqlEngine = create_engine(conn_string, pool_recycle = 3600)
connection = sqlEngine.connect()
dim_customers.to_sql('dim_customers', connection, index=False, if_exists='replace')
sqlEngine.execute(f"ALTER TABLE {'dim_customers'} ADD PRIMARY KEY ({'Invoice_Key'});")
connection.close()

In [ ]:
# inserting dim_artstats into chinook_dw in sql and declaring pk

In [106]:
conn_string = f'mysql+pymysql://{user_id}:{pwd}@{host_name}/{dw_name}'
sqlEngine = create_engine(conn_string, pool_recycle = 3600)
connection = sqlEngine.connect()
dim_artstats.to_sql('dim_artstats', connection, index=False, if_exists='replace')
sqlEngine.execute(f"ALTER TABLE {'dim_artStats'} ADD PRIMARY KEY ({'Album_Key'});")
connection.close()

In [ ]:
# inserting dim_alb_track into chinook_dw in sql and declaring pk

In [108]:
conn_string = f'mysql+pymysql://{user_id}:{pwd}@{host_name}/{dw_name}'
sqlEngine = create_engine(conn_string, pool_recycle = 3600)
connection = sqlEngine.connect()
dim_alb_track.to_sql('dim_alb_track', connection, index=False, if_exists='replace')
sqlEngine.execute(f"ALTER TABLE {'dim_alb_track'} ADD PRIMARY KEY ({'Track_Key'});")
connection.close()

In [13]:
# verify that dim_customers is in chinook_dw in sql

In [122]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
df_dim_customers = pd.read_sql("SELECT * FROM dim_customers order by Invoice_Key;", conn)
conn.close()
df_dim_customers.head()

,Invoice_Key,Customer_Key,InvoiceDate,BillingCity,BillingCountry,BillingPostalCode,Total,FirstName,LastName,Company,Phone,Fax,Email,SupportRepId
0,98,1,2010-03-11,São José dos Campos,Brazil,12227-000,3.98,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,121,1,2010-06-13,São José dos Campos,Brazil,12227-000,3.96,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,143,1,2010-09-15,São José dos Campos,Brazil,12227-000,5.94,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,195,1,2011-05-06,São José dos Campos,Brazil,12227-000,0.99,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
4,316,1,2012-10-27,São José dos Campos,Brazil,12227-000,1.98,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3


In [ ]:
# verify that dim_artstats is in chinook_dw in sql

In [68]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
df_dim_artstats = pd.read_sql("SELECT * FROM dim_artstats order by Album_Key;", conn)
conn.close()
df_dim_artstats.head()

,Album_Key,Title,Artist_Key,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith


In [ ]:
# verify that dim_alb_track is in chinook_dw in sql

In [109]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
df_dim_alb_track = pd.read_sql("SELECT * FROM dim_alb_track order by Track_Key;", conn)
conn.close()
df_dim_alb_track.head()

,Track_Key,Name,Album_Key,MediaType_Key,Genre_Key,Composer,Milliseconds,Bytes,UnitPrice,Title,Artist_Key
0,1,For Those About To Rock (We Salute You) - TEST,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Restless and Wild,2
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Restless and Wild,2
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Restless and Wild,2


In [ ]:
# DATE DIMENSION, FACT TABLE

In [124]:
# create the date keys
date_list = []
counter = 0

if df_invoice_fct['InvoiceDate'][0] == df_invoice_fct['InvoiceDate'][1]:
    counter = counter
    date_list.append(counter)
else:
    counter += 1
    date_list.append(counter)

for i in range(1, len(df_invoice_fct)-1):
    if df_invoice_fct['InvoiceDate'][i] == df_invoice_fct['InvoiceDate'][i+1]:
        counter == counter
        date_list.append(counter)
    else:
        counter += 1
        date_list.append(counter)

date_list.insert(0, 0)
# add the date keys as a column into the fact table
df_invoice_fct['DateKey'] = date_list

# drop the date column 
df_invoice_fct.drop(['InvoiceDate'], axis=1, inplace=True)
# final fact table with new date key column (date dimension)
df_invoice_fct.head(30)

,InvoiceId,CustomerId,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,DateKey
0,1,2,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,0
1,2,4,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,1
2,3,8,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,2
3,4,14,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,3
4,5,23,69 Salem Street,Boston,MA,USA,2113,13.86,4
5,6,37,Berger Straße 10,Frankfurt,None,Germany,60316,0.99,5
6,7,38,Barbarossastraße 19,Berlin,None,Germany,10779,1.98,6
7,8,40,"8, Rue Hanovre",Paris,None,France,75002,1.98,6
8,9,42,"9, Place Louis Barthou",Bordeaux,None,France,33000,3.96,7
9,10,46,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94,8


In [ ]:
# inserting final fact table into sql and declaring pk

In [126]:
invoice_fact = df_invoice_fct
conn_string = f'mysql+pymysql://{user_id}:{pwd}@{host_name}/{dw_name}'
sqlEngine = create_engine(conn_string, pool_recycle = 3600)
connection = sqlEngine.connect()
invoice_fact.to_sql('invoice_fact', connection, index=False, if_exists='replace')
sqlEngine.execute(f"ALTER TABLE {'invoice_fact'} ADD PRIMARY KEY ({'InvoiceId'});")
connection.close()

In [ ]:
# verify that fact table is in chinook_dw in sql

In [127]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
invoice_fact = pd.read_sql("SELECT * FROM invoice_fact order by InvoiceId;", conn)
conn.close()
invoice_fact.head()

,InvoiceId,CustomerId,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,DateKey
0,1,2,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,0
1,2,4,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,1
2,3,8,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,2
3,4,14,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,3
4,5,23,69 Salem Street,Boston,MA,USA,2113,13.86,4


In [15]:
# QUERIES

In [16]:
# total sales across different countries:

In [17]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
df_sales = pd.read_sql("select sum(Total),BillingCountry from chinook_dw.dim_customers group by BillingCountry;", conn)
conn.close()
df_sales.head()

,sum(Total),BillingCountry
0,190.10,Brazil
1,156.48,Germany
2,303.96,Canada
3,39.62,Norway
4,90.24,Czech Republic


In [ ]:
# top 3 customers based on total sales:

In [135]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
df_customer_sales = pd.read_sql("select sum(Total),CustomerId from chinook_dw.invoice_fact group by CustomerId order by sum(Total) desc;", conn)
conn.close()
df_customer_sales.head(3)

,sum(Total),CustomerId
0,49.62,6
1,47.62,26
2,46.62,57


In [ ]:
# average song length based on composer:

In [136]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=dw_name)
df_length = pd.read_sql("select avg(Milliseconds),Composer from chinook_dw.dim_alb_track group by Composer;", conn)
conn.close()
df_length.head()

,avg(Milliseconds),Composer
0,240041.5,"Angus Young, Malcolm Young, Brian Johnson"
1,342562.0,None
2,230619.0,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,252051.0,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,375418.0,Deaffy & R.A. Smith-Diesel
